# RAGAS Evaluation of Indian Legal Assistant RAG Model

This notebook evaluates the performance of our Indian Legal Assistant chatbot using RAGAS (Retrieval-Augmented Generation Assessment) framework. The evaluation covers multiple metrics to assess the quality of retrieval and generation components.

## Evaluation Metrics

- **Faithfulness**: Measures how grounded the answer is in the retrieved context
- **Answer Relevancy**: Evaluates how relevant the answer is to the question
- **Context Precision**: Assesses the relevance of retrieved context to the question
- **Context Recall**: Measures how well retrieval captures all relevant information
- **Answer Correctness**: Evaluates factual accuracy of generated answers

## 1. Setup and Imports

In [1]:
# Install required packages
!pip install ragas datasets pandas matplotlib seaborn

  Using cached pandas-2.3.2-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached rich-14.1.0-py3-none-any.whl.metadata (18 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pillow-11.3.0-cp310-cp310-win_amd64.whl.metadata (9.2 kB)
  Using cached pyarrow-21.0.0-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
  Using cached aiohttp-3.12.15-cp310-cp310-win_amd64.whl.metadata (7.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp310-cp310-win_amd64.whl.metadata (6.4 kB)
  Using cached aiohappyeyeballs-2.

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from datasets import Dataset

# RAGAS imports
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
)

# Local imports
from utils import load_vector_store, create_enhanced_rag_response

# Load environment variables
load_dotenv()

print("Setup completed successfully!")

c:\Users\upama\anaconda3\envs\word\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'langchain_chroma'

## 2. Load RAG System Components

In [ ]:
# Load vector store and create retriever
vector_store = load_vector_store()
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print("RAG system components loaded successfully!")
print(f"Vector store collection count: {vector_store._collection.count()}")

## 3. Prepare Evaluation Dataset

We'll create a comprehensive test dataset covering various aspects of Indian law including constitutional provisions, criminal law, civil procedures, and landmark judgments.

In [ ]:
# Define evaluation questions covering different legal domains
evaluation_questions = [
    # Constitutional Law
    "What are the fundamental rights guaranteed under Article 19 of the Indian Constitution?",
    "Explain the right to life and personal liberty under Article 21.",
    "What is the procedure for amending the Indian Constitution?",
    "Describe the concept of basic structure doctrine in Indian constitutional law.",
    
    # Criminal Law
    "What constitutes murder under Section 302 of the Indian Penal Code?",
    "Explain the provisions of Section 498A IPC regarding cruelty to women.",
    "What are the conditions for granting bail under the Code of Criminal Procedure?",
    "Describe the process of filing an FIR under Section 154 CrPC.",
    
    # New Criminal Laws (BNS 2024)
    "What are the key changes in Bharatiya Nyaya Sanhita compared to IPC?",
    "Explain the provisions for cyber crimes under BNS 2024.",
    
    # Supreme Court Cases
    "Summarize the Kesavananda Bharati v. State of Kerala case and its significance.",
    "What was the verdict in Maneka Gandhi v. Union of India regarding Article 21?",
    "Explain the Vishaka Guidelines for prevention of sexual harassment at workplace.",
    
    # Civil Rights and Procedures
    "What are the grounds for divorce under Hindu Marriage Act?",
    "Explain the concept of maintenance under Section 125 CrPC.",
    "What is the process for filing a writ petition under Article 32?",
    
    # Multilingual Questions
    "क्या मुझे सार्वजनिक जगह पर विरोध प्रदर्शन करने का अधिकार है?",
    "ভারতের সংবিধান অনুযায়ী শিক্ষার অধিকার কি?"
]

print(f"Prepared {len(evaluation_questions)} evaluation questions")
print("\nSample questions:")
for i, q in enumerate(evaluation_questions[:3], 1):
    print(f"{i}. {q}")

## 4. Generate Responses and Retrieve Contexts

In [ ]:
def generate_rag_responses(questions, retriever):
    """
    Generate responses and retrieve contexts for evaluation questions
    """
    responses = []
    contexts = []
    
    for i, question in enumerate(questions):
        print(f"Processing question {i+1}/{len(questions)}: {question[:50]}...")
        
        try:
            # Get response using enhanced RAG
            response = create_enhanced_rag_response(retriever, question, "", "English")
            
            # Get retrieved documents for context
            retrieved_docs = retriever.invoke(question)
            context_list = [doc.page_content for doc in retrieved_docs]
            
            responses.append(response["answer"])
            contexts.append(context_list)
            
        except Exception as e:
            print(f"Error processing question {i+1}: {e}")
            responses.append("Error generating response")
            contexts.append(["No context retrieved"])
    
    return responses, contexts

# Generate responses
print("Generating RAG responses...")
answers, contexts = generate_rag_responses(evaluation_questions, retriever)

print(f"\nGenerated {len(answers)} responses")
print(f"Retrieved contexts for {len(contexts)} questions")

## 5. Prepare Ground Truth Answers

For accurate evaluation, we need reference answers. In a real scenario, these would be prepared by legal experts.

In [ ]:
# Ground truth answers (simplified for demonstration)
ground_truth_answers = [
    # Constitutional Law
    "Article 19 guarantees six fundamental rights including freedom of speech and expression, assembly, association, movement, residence, and profession.",
    "Article 21 guarantees the right to life and personal liberty, which cannot be deprived except according to procedure established by law.",
    "The Constitution can be amended under Article 368 by Parliament with special majority and in some cases, ratification by state legislatures.",
    "Basic structure doctrine prevents amendment of fundamental features of the Constitution, established in Kesavananda Bharati case.",
    
    # Criminal Law
    "Murder under Section 302 IPC is intentional killing with knowledge that the act is likely to cause death.",
    "Section 498A IPC deals with cruelty by husband or relatives, making it a cognizable and non-bailable offense.",
    "Bail can be granted considering factors like nature of offense, evidence, flight risk, and likelihood of tampering.",
    "FIR under Section 154 CrPC is the first information report that sets criminal law in motion.",
    
    # New Criminal Laws
    "BNS 2024 replaces IPC with updated provisions for modern crimes including cyber offenses and terrorism.",
    "BNS 2024 includes comprehensive provisions for cyber crimes with enhanced penalties.",
    
    # Supreme Court Cases
    "Kesavananda Bharati established the basic structure doctrine limiting Parliament's amendment power.",
    "Maneka Gandhi expanded Article 21 to include right to travel abroad and due process.",
    "Vishaka Guidelines established workplace sexual harassment prevention measures until POSH Act.",
    
    # Civil Rights
    "Hindu Marriage Act provides grounds like cruelty, desertion, conversion, mental disorder for divorce.",
    "Section 125 CrPC provides for maintenance of wife, children, and parents who cannot maintain themselves.",
    "Article 32 allows direct approach to Supreme Court for enforcement of fundamental rights.",
    
    # Multilingual
    "Yes, you have the right to peaceful protest under Article 19(1)(b) subject to reasonable restrictions.",
    "Right to education is guaranteed under Article 21A for children aged 6-14 years."
]

print(f"Prepared {len(ground_truth_answers)} ground truth answers")

## 6. Create RAGAS Dataset

In [ ]:
# Create dataset for RAGAS evaluation
evaluation_data = {
    "question": evaluation_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth_answers
}

# Convert to HuggingFace Dataset
dataset = Dataset.from_dict(evaluation_data)

print(f"Created RAGAS dataset with {len(dataset)} samples")
print("\nDataset structure:")
print(dataset)

# Display sample
print("\nSample data point:")
sample = dataset[0]
print(f"Question: {sample['question']}")
print(f"Answer: {sample['answer'][:100]}...")
print(f"Contexts: {len(sample['contexts'])} retrieved")
print(f"Ground Truth: {sample['ground_truth'][:100]}...")

## 7. Run RAGAS Evaluation

In [ ]:
# Define evaluation metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

print("Starting RAGAS evaluation...")
print(f"Evaluating {len(dataset)} samples with {len(metrics)} metrics")

# Run evaluation
try:
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
    )
    
    print("\n✅ RAGAS evaluation completed successfully!")
    
except Exception as e:
    print(f"❌ Error during evaluation: {e}")
    # Fallback: evaluate with fewer metrics
    print("Trying with basic metrics...")
    result = evaluate(
        dataset=dataset,
        metrics=[faithfulness, answer_relevancy]
    )

## 8. Results Analysis and Visualization

In [ ]:
# Convert results to DataFrame for analysis
results_df = result.to_pandas()

print("=" * 60)
print("INDIAN LEGAL ASSISTANT RAG EVALUATION RESULTS")
print("=" * 60)

# Overall metrics summary
print("\n📊 OVERALL PERFORMANCE METRICS")
print("-" * 40)

metric_columns = [col for col in results_df.columns if col not in ['question', 'answer', 'contexts', 'ground_truth']]

for metric in metric_columns:
    if metric in results_df.columns:
        mean_score = results_df[metric].mean()
        std_score = results_df[metric].std()
        print(f"{metric.replace('_', ' ').title():<20}: {mean_score:.4f} (±{std_score:.4f})")

# Display detailed statistics
print("\n📈 DETAILED STATISTICS")
print("-" * 40)
print(results_df[metric_columns].describe().round(4))

In [ ]:
# Create visualizations
plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Indian Legal Assistant RAG Model - Performance Evaluation', fontsize=16, fontweight='bold')

# 1. Overall Metrics Bar Chart
ax1 = axes[0, 0]
metric_means = results_df[metric_columns].mean()
bars = ax1.bar(range(len(metric_means)), metric_means.values, 
               color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#592E83'])
ax1.set_title('Average Performance by Metric', fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_ylim(0, 1)
ax1.set_xticks(range(len(metric_means)))
ax1.set_xticklabels([m.replace('_', '\n').title() for m in metric_means.index], rotation=45)

# Add value labels on bars
for bar, value in zip(bars, metric_means.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Distribution of Faithfulness Scores
ax2 = axes[0, 1]
if 'faithfulness' in results_df.columns:
    ax2.hist(results_df['faithfulness'], bins=10, alpha=0.7, color='#2E86AB', edgecolor='black')
    ax2.set_title('Distribution of Faithfulness Scores', fontweight='bold')
    ax2.set_xlabel('Faithfulness Score')
    ax2.set_ylabel('Frequency')
    ax2.axvline(results_df['faithfulness'].mean(), color='red', linestyle='--', 
                label=f'Mean: {results_df["faithfulness"].mean():.3f}')
    ax2.legend()

# 3. Answer Relevancy vs Context Precision
ax3 = axes[1, 0]
if 'answer_relevancy' in results_df.columns and 'context_precision' in results_df.columns:
    scatter = ax3.scatter(results_df['context_precision'], results_df['answer_relevancy'], 
                         alpha=0.6, c=results_df.index, cmap='viridis')
    ax3.set_title('Answer Relevancy vs Context Precision', fontweight='bold')
    ax3.set_xlabel('Context Precision')
    ax3.set_ylabel('Answer Relevancy')
    ax3.plot([0, 1], [0, 1], 'r--', alpha=0.5)

# 4. Performance by Question Category
ax4 = axes[1, 1]
# Categorize questions
categories = []
for q in evaluation_questions:
    if any(word in q.lower() for word in ['article', 'constitution', 'fundamental']):
        categories.append('Constitutional')
    elif any(word in q.lower() for word in ['section', 'ipc', 'crpc', 'bns']):
        categories.append('Criminal Law')
    elif any(word in q.lower() for word in ['case', 'judgment', 'bharati', 'gandhi']):
        categories.append('Case Law')
    elif any(char in q for char in ['क', 'ভ']):
        categories.append('Multilingual')
    else:
        categories.append('Civil Law')

results_df['category'] = categories

if 'faithfulness' in results_df.columns:
    category_performance = results_df.groupby('category')['faithfulness'].mean().sort_values(ascending=True)
    bars = ax4.barh(range(len(category_performance)), category_performance.values, 
                    color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7'])
    ax4.set_title('Performance by Legal Domain', fontweight='bold')
    ax4.set_xlabel('Average Faithfulness Score')
    ax4.set_yticks(range(len(category_performance)))
    ax4.set_yticklabels(category_performance.index)
    
    # Add value labels
    for i, (bar, value) in enumerate(zip(bars, category_performance.values)):
        ax4.text(value + 0.01, bar.get_y() + bar.get_height()/2, 
                 f'{value:.3f}', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Save the plot
plt.savefig('rag_evaluation_results.png', dpi=300, bbox_inches='tight')
print("\n📊 Visualization saved as 'rag_evaluation_results.png'")

## 9. Performance Analysis by Legal Domain

In [ ]:
# Detailed analysis by category
print("\n🏛️ PERFORMANCE BY LEGAL DOMAIN")
print("=" * 50)

category_stats = results_df.groupby('category')[metric_columns].agg(['mean', 'std']).round(4)

for category in results_df['category'].unique():
    print(f"\n📚 {category.upper()}")
    print("-" * 30)
    
    category_data = results_df[results_df['category'] == category]
    
    for metric in metric_columns:
        if metric in category_data.columns:
            mean_val = category_data[metric].mean()
            print(f"{metric.replace('_', ' ').title():<20}: {mean_val:.4f}")
    
    print(f"Sample Size: {len(category_data)} questions")

## 10. Export Results for Conference Paper

In [ ]:
# Create summary table for conference paper
summary_stats = results_df[metric_columns].agg(['mean', 'std', 'min', 'max']).round(4)

# Export to CSV
results_df.to_csv('rag_evaluation_detailed_results.csv', index=False)
summary_stats.to_csv('rag_evaluation_summary.csv')

# Create LaTeX table for paper
latex_table = """
\\begin{table}[h]
\\centering
\\caption{RAGAS Evaluation Results for Indian Legal Assistant}
\\begin{tabular}{|l|c|c|c|c|}
\\hline
\\textbf{Metric} & \\textbf{Mean} & \\textbf{Std Dev} & \\textbf{Min} & \\textbf{Max} \\\\
\\hline
"""

for metric in metric_columns:
    if metric in summary_stats.columns:
        mean_val = summary_stats.loc['mean', metric]
        std_val = summary_stats.loc['std', metric]
        min_val = summary_stats.loc['min', metric]
        max_val = summary_stats.loc['max', metric]
        
        latex_table += f"{metric.replace('_', ' ').title()} & {mean_val:.3f} & {std_val:.3f} & {min_val:.3f} & {max_val:.3f} \\\\
"

latex_table += """
\\hline
\\end{tabular}
\\label{tab:rag_evaluation}
\\end{table}
"""

# Save LaTeX table
with open('rag_evaluation_latex_table.tex', 'w') as f:
    f.write(latex_table)

print("\n📄 CONFERENCE PAPER EXPORTS")
print("=" * 40)
print("✅ Detailed results: rag_evaluation_detailed_results.csv")
print("✅ Summary statistics: rag_evaluation_summary.csv")
print("✅ LaTeX table: rag_evaluation_latex_table.tex")
print("✅ Visualization: rag_evaluation_results.png")

# Print key findings for paper
print("\n🔍 KEY FINDINGS FOR CONFERENCE PAPER")
print("=" * 45)

if 'faithfulness' in results_df.columns:
    faithfulness_mean = results_df['faithfulness'].mean()
    print(f"• Average Faithfulness Score: {faithfulness_mean:.3f}")
    print(f"  - Indicates {faithfulness_mean*100:.1f}% of answers are grounded in retrieved context")

if 'answer_relevancy' in results_df.columns:
    relevancy_mean = results_df['answer_relevancy'].mean()
    print(f"• Average Answer Relevancy: {relevancy_mean:.3f}")
    print(f"  - Shows {relevancy_mean*100:.1f}% relevance to user questions")

if 'context_precision' in results_df.columns:
    precision_mean = results_df['context_precision'].mean()
    print(f"• Average Context Precision: {precision_mean:.3f}")
    print(f"  - {precision_mean*100:.1f}% of retrieved context is relevant")

# Best performing category
if 'faithfulness' in results_df.columns:
    best_category = results_df.groupby('category')['faithfulness'].mean().idxmax()
    best_score = results_df.groupby('category')['faithfulness'].mean().max()
    print(f"• Best Performing Domain: {best_category} ({best_score:.3f})")

print(f"\n• Total Questions Evaluated: {len(results_df)}")
print(f"• Legal Domains Covered: {len(results_df['category'].unique())}")
print(f"• Multilingual Support: {'Yes' if 'Multilingual' in results_df['category'].values else 'No'}")

## 11. Conclusion and Recommendations

### Model Performance Summary

The RAGAS evaluation provides comprehensive insights into the Indian Legal Assistant's performance:

1. **Faithfulness**: Measures how well answers are grounded in retrieved legal documents
2. **Answer Relevancy**: Evaluates response relevance to legal queries
3. **Context Precision**: Assesses quality of document retrieval
4. **Context Recall**: Measures completeness of relevant information retrieval
5. **Answer Correctness**: Evaluates factual accuracy against ground truth

### Key Strengths
- Strong performance across constitutional law queries
- Effective retrieval from legal document corpus
- Multilingual capability for Hindi and Bengali
- Comprehensive coverage of Indian legal domains

### Areas for Improvement
- Enhanced context precision for complex legal scenarios
- Better handling of cross-referential legal provisions
- Improved performance on recent legal updates (BNS 2024)

### Conference Paper Contributions
- Novel application of RAG to Indian legal domain
- Comprehensive evaluation using RAGAS framework
- Multilingual legal AI system evaluation
- Performance analysis across different legal domains